<a href="https://colab.research.google.com/github/jeffreyfeng99/SYDE_522_A3/blob/master/SYDE_522_Assignment_3_refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Submission Notes
Group members:

Jeffrey Feng, 20704800

Joey Kuang, 20726074

# Import statements

In [1]:
import os
import pandas as pd
import numpy as np
import random
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Function
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torchvision import datasets
from torchvision import models
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Config

In [3]:
cuda = True
cudnn.benchmark = True
LR = 1e-3
BATCH_SIZE = 64
IMAGE_SIZE = 224 #227
FT_OUT_SIZE = 512
N_EPOCH = 10 #100


# Data loader

In [4]:
dataset_root = '/content/drive/MyDrive/4B/SYDE-522/data'
source_dataset_name = 'train_set'
target_dataset_name = 'test_set'

source_image_root = os.path.join(dataset_root, source_dataset_name)
target_image_root = os.path.join(dataset_root, target_dataset_name)

train_label_list = os.path.join(dataset_root, 'train_labels.csv')
test_label_list = os.path.join(dataset_root, 'test_labels.csv')

In [5]:
class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list=None, transform=None):
        self.root = data_root
        self.transform = transform

        # we only pass data_list if it's training set
        if data_list is not None:
            df = pd.read_csv(data_list)
            self.img_paths = df['dir'].to_list()

            if 'label2' in df.columns:
                self.img_labels = df['label2'].to_list()
            else: 
                self.img_labels = ['0' for i in range(len(self.img_paths))]
        else:
            # Walk through test folder - we don't need labels
            self.img_paths = [f for root,dirs,files in os.walk(data_root) for f in files if f.endswith('.png')]
            self.img_labels = ['0' for i in range(len(self.img_paths))]

        self.n_data = len(self.img_paths)

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item%self.n_data], self.img_labels[item%self.n_data]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels

    def __len__(self):
        return self.n_data

In [6]:
# Preprocess data
def preprocess_fn(mu=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)):
  img_transform = transforms.Compose([
      transforms.Resize(IMAGE_SIZE),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) # why isn't this (0.1307,) and (0.3081,) like in test?
  ])

  return img_transform

def prep_dataloader(image_root, label_list=None, img_transform=None):
    dataset = GetLoader(
        data_root=image_root,
        data_list=label_list,
        transform=img_transform
    )

    dataloader = data.DataLoader(
        dataset=dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8,
        drop_last=True)
    
    return dataset, dataloader


# Model definition

In [7]:
# if False, then we are feature extracting
def set_parameter_requires_grad(model, finetune):
    for param in model.parameters():
        param.requires_grad = finetune

In [8]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None


class CNNModel(nn.Module):

    def __init__(self, model_name="resnet18"):
        super(CNNModel, self).__init__()

        if model_name == "resnet18":
            self.feature = models.resnet18(pretrained=True) 
            self.feature.fc = nn.Identity()
        elif model_name == "vgg11":
            self.feature = models.vgg11_bn(pretrained=True) 
            self.feature.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1)) # original is (7,7)
            self.feature.classifier = nn.Identity()
        else:
            # Need some default model?
            self.feature = nn.Sequential()
            self.feature.add_module('f_conv1', nn.Conv2d(3, 64, kernel_size=5))
            self.feature.add_module('f_bn1', nn.BatchNorm2d(64))
            self.feature.add_module('f_pool1', nn.MaxPool2d(2))
            self.feature.add_module('f_relu1', nn.ReLU(True))
            self.feature.add_module('f_conv2', nn.Conv2d(64, 128, kernel_size=3))
            self.feature.add_module('f_bn2', nn.BatchNorm2d(128))
            self.feature.add_module('f_drop1', nn.Dropout2d())
            self.feature.add_module('f_pool2', nn.MaxPool2d(2))
            self.feature.add_module('f_relu2', nn.ReLU(True))
            self.feature.add_module('f_conv3', nn.Conv2d(128, 256, kernel_size=3))
            self.feature.add_module('f_bn3', nn.BatchNorm2d(256))
            self.feature.add_module('f_drop3', nn.Dropout2d())
            self.feature.add_module('f_pool3', nn.MaxPool2d(2))
            self.feature.add_module('f_relu4', nn.ReLU(True))
            self.feature.add_module('f_conv4', nn.Conv2d(256, 256, kernel_size=3))
            self.feature.add_module('f_bn4', nn.BatchNorm2d(256))
            self.feature.add_module('f_drop4', nn.Dropout2d())
            self.feature.add_module('f_pool4', nn.MaxPool2d(2))
            self.feature.add_module('f_relu4', nn.ReLU(True))
            self.feature.add_module('f_conv5', nn.Conv2d(256, 512, kernel_size=3))
            self.feature.add_module('f_bn5', nn.BatchNorm2d(512))
            self.feature.add_module('f_drop5', nn.Dropout2d())
            self.feature.add_module('f_pool5', nn.MaxPool2d(2))
            self.feature.add_module('f_relu5', nn.ReLU(True))

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(FT_OUT_SIZE, 100))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout2d())
        self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(100, 7))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax(dim=1))

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(FT_OUT_SIZE, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 4))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, IMAGE_SIZE, IMAGE_SIZE)
        feature = self.feature(input_data)
        feature = feature.view(-1, FT_OUT_SIZE)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

# Test pipeline

In [11]:
def test(net, dataset_name, epoch):

    # image_root = os.path.join(dataset_root, dataset_name)

    # load data
    img_transform_source = preprocess_fn(mu=(0.1307,), std=(0.3081,))
    img_transform_target = preprocess_fn()

    dataset_source, dataloader_source = prep_dataloader(
        image_root=os.path.join(source_image_root, 'train_set'),
        label_list=train_label_list,
        img_transform=img_transform_source
    )

    dataset_target, dataloader_target = prep_dataloader(
        image_root=os.path.join(target_image_root, 'test_set'),
        img_transform=img_transform_target
    )

    """ testing """

    # my_net = torch.load(os.path.join('output', 'model_epoch_' + str(epoch) + '.pth'))
    my_net = net.eval() #my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader_source)
    data_target_iter = iter(dataloader_source)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label) # TODO: why

        input_img = torch.FloatTensor(batch_size, 3, IMAGE_SIZE, IMAGE_SIZE)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)

        class_output, _ = my_net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))

# Training pipeline

In [12]:
manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# load data
img_transform_source = preprocess_fn()
img_transform_target = preprocess_fn()

dataset_source, dataloader_source = prep_dataloader(
    image_root=os.path.join(source_image_root, 'train_set'), # TODO should we unnest
    label_list=train_label_list,
    img_transform=img_transform_source
)

dataset_target, dataloader_target = prep_dataloader(
    image_root=os.path.join(target_image_root, 'test_set'),
    img_transform=img_transform_target
)

# load model
my_net = CNNModel()

# setup optimizer
optimizer = optim.Adam(my_net.parameters(), lr=LR)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

set_parameter_requires_grad(my_net, True)

# training
for epoch in tqdm(range(N_EPOCH)):

    len_dataloader = max(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / N_EPOCH / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = data_source_iter.next()
        s_img, s_label = data_source

        my_net.zero_grad()
        batch_size = len(s_label)

        input_img = torch.FloatTensor(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE)
        class_label = torch.LongTensor(BATCH_SIZE)
        domain_label = torch.zeros(BATCH_SIZE)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)

        class_output, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        if i == len(dataloader_target):
            data_target_iter = iter(dataloader_target)
        data_target = data_target_iter.next()
        t_img, _ = data_target

        batch_size = len(t_img) # TODO: why?

        input_img = torch.FloatTensor(batch_size, 3, IMAGE_SIZE, IMAGE_SIZE)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)

        _, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        i += 1

        print('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
            % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
                err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

    # torch.save(my_net, 'output/model_epoch_{}.pth'.format(epoch))
    test(my_net, source_dataset_name, epoch)
    my_net.train()

print('done')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0, [iter: 1 / all 94], err_s_label: 1.997244, err_s_domain: 1.568725, err_t_domain: 1.130663
epoch: 0, [iter: 2 / all 94], err_s_label: 1.765900, err_s_domain: 1.518329, err_t_domain: 1.094720
epoch: 0, [iter: 3 / all 94], err_s_label: 1.686886, err_s_domain: 1.502687, err_t_domain: 1.055828
epoch: 0, [iter: 4 / all 94], err_s_label: 1.626349, err_s_domain: 1.450271, err_t_domain: 1.033716
epoch: 0, [iter: 5 / all 94], err_s_label: 1.581231, err_s_domain: 1.428987, err_t_domain: 0.985104
epoch: 0, [iter: 6 / all 94], err_s_label: 1.396519, err_s_domain: 1.375621, err_t_domain: 0.951195
epoch: 0, [iter: 7 / all 94], err_s_label: 1.358490, err_s_domain: 1.342863, err_t_domain: 0.927840
epoch: 0, [iter: 8 / all 94], err_s_label: 1.405157, err_s_domain: 1.315300, err_t_domain: 0.886972
epoch: 0, [iter: 9 / all 94], err_s_label: 1.284019, err_s_domain: 1.267387, err_t_domain: 0.890374
epoch: 0, [iter: 10 / all 94], err_s_label: 1.240239, err_s_domain: 1.261973, err_t_domain: 0.856447

 10%|█         | 1/10 [02:14<20:12, 134.69s/it]

epoch: 0, accuracy of the train_set dataset: 0.595745
epoch: 1, [iter: 1 / all 94], err_s_label: 0.585238, err_s_domain: 0.683288, err_t_domain: 0.683595
epoch: 1, [iter: 2 / all 94], err_s_label: 0.605381, err_s_domain: 0.709004, err_t_domain: 0.686824
epoch: 1, [iter: 3 / all 94], err_s_label: 0.456547, err_s_domain: 0.691447, err_t_domain: 0.684696
epoch: 1, [iter: 4 / all 94], err_s_label: 0.348675, err_s_domain: 0.672196, err_t_domain: 0.686050
epoch: 1, [iter: 5 / all 94], err_s_label: 0.469285, err_s_domain: 0.699771, err_t_domain: 0.674607
epoch: 1, [iter: 6 / all 94], err_s_label: 0.669177, err_s_domain: 0.690303, err_t_domain: 0.675396
epoch: 1, [iter: 7 / all 94], err_s_label: 0.439871, err_s_domain: 0.704488, err_t_domain: 0.677584
epoch: 1, [iter: 8 / all 94], err_s_label: 0.384647, err_s_domain: 0.681420, err_t_domain: 0.697065
epoch: 1, [iter: 9 / all 94], err_s_label: 0.566749, err_s_domain: 0.694318, err_t_domain: 0.687810
epoch: 1, [iter: 10 / all 94], err_s_label: 0.

 20%|██        | 2/10 [04:28<17:51, 133.98s/it]

...

epoch: 7, accuracy of the train_set dataset: 0.766622
epoch: 8, [iter: 1 / all 94], err_s_label: 0.080252, err_s_domain: 0.702440, err_t_domain: 0.683516
epoch: 8, [iter: 2 / all 94], err_s_label: 0.197884, err_s_domain: 0.732416, err_t_domain: 0.681200
epoch: 8, [iter: 3 / all 94], err_s_label: 0.071448, err_s_domain: 0.719256, err_t_domain: 0.664511
epoch: 8, [iter: 4 / all 94], err_s_label: 0.078125, err_s_domain: 0.722306, err_t_domain: 0.666563
epoch: 8, [iter: 5 / all 94], err_s_label: 0.133891, err_s_domain: 0.731457, err_t_domain: 0.655362
epoch: 8, [iter: 6 / all 94], err_s_label: 0.142733, err_s_domain: 0.732756, err_t_domain: 0.654106
epoch: 8, [iter: 7 / all 94], err_s_label: 0.125604, err_s_domain: 0.740691, err_t_domain: 0.650261
epoch: 8, [iter: 8 / all 94], err_s_label: 0.510376, err_s_domain: 0.740912, err_t_domain: 0.649183
epoch: 8, [iter: 9 / all 94], err_s_label: 0.187949, err_s_domain: 0.740934, err_t_domain: 0.645924
epoch: 8, [iter: 10 / all 94], err_s_label: 0.

 90%|█████████ | 9/10 [20:09<02:14, 134.74s/it]

epoch: 8, accuracy of the train_set dataset: 0.915559
epoch: 9, [iter: 1 / all 94], err_s_label: 0.041206, err_s_domain: 0.690931, err_t_domain: 0.708451
epoch: 9, [iter: 2 / all 94], err_s_label: 0.127195, err_s_domain: 0.690014, err_t_domain: 0.697327
epoch: 9, [iter: 3 / all 94], err_s_label: 0.049819, err_s_domain: 0.683567, err_t_domain: 0.703098
epoch: 9, [iter: 4 / all 94], err_s_label: 0.097352, err_s_domain: 0.702248, err_t_domain: 0.694721
epoch: 9, [iter: 5 / all 94], err_s_label: 0.055993, err_s_domain: 0.702008, err_t_domain: 0.694088
epoch: 9, [iter: 6 / all 94], err_s_label: 0.084273, err_s_domain: 0.706308, err_t_domain: 0.689862
epoch: 9, [iter: 7 / all 94], err_s_label: 0.108321, err_s_domain: 0.713219, err_t_domain: 0.691112
epoch: 9, [iter: 8 / all 94], err_s_label: 0.200248, err_s_domain: 0.707714, err_t_domain: 0.681925
epoch: 9, [iter: 9 / all 94], err_s_label: 0.107062, err_s_domain: 0.717628, err_t_domain: 0.687339
epoch: 9, [iter: 10 / all 94], err_s_label: 0.

100%|██████████| 10/10 [22:25<00:00, 134.58s/it]

epoch: 9, accuracy of the train_set dataset: 0.967420
done
